<a href="https://colab.research.google.com/github/lolotta/SleepTechWordDetection/blob/master/Sleep_HTR_david_doesnt_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#Download the IAM online Dataset

!wget -q https://git.io/J0fjL -O IAM_Words.zip
!unzip -qq IAM_Words.zip
!mkdir data
!mkdir data/words
!tar -xf IAM_Words/words.tgz -C data/words
!mv IAM_Words/words.txt data

^C
[IAM_Words.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of IAM_Words.zip or
        IAM_Words.zip.zip, and cannot find IAM_Words.zip.ZIP, period.
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/words’: File exists
mv: cannot stat 'IAM_Words/words.txt': No such file or directory


In [29]:
pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 563.7 MB 12 kB/s 
     |████████████████████████████████| 439 kB 82.4 MB/s 
     |████████████████████████████████| 1.7 MB 65.5 MB/s 
     |████████████████████████████████| 5.7 MB 59.3 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.12.6 which is incompatible.


In [46]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import datetime
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras
from keras.layers import Dense, Conv2D, AveragePooling2D, TimeDistributed, LSTM, GlobalAveragePooling2D, AbstractRNNCell, MaxPooling2D, Reshape, Dropout
from keras.initializers import Orthogonal
import tqdm

#Make a list of all the words without Comments or errored words
base_path = "data"
words_list = []

words = open(f"{base_path}/words.txt", "r").readlines()
for line in words:
    if line[0] == "#":
        continue
    if line.split(" ")[1] != "err":  # We don't need to deal with errored entries.
        words_list.append(line)

#Shuffle the word list
np.random.shuffle(words_list)

#Split the data into train, test, validation
split_idx = int(0.9 * len(words_list))
train_samples = words_list[:split_idx]
test_samples = words_list[split_idx:]

val_split_idx = int(0.5 * len(test_samples))
validation_samples = test_samples[:val_split_idx]
test_samples = test_samples[val_split_idx:]

#Check if diving was clean (no data was left out)
assert len(words_list) == len(train_samples) + len(validation_samples) + len(
    test_samples
)

print(f"Total training samples: {len(train_samples)}")
print(f"Total validation samples: {len(validation_samples)}")
print(f"Total test samples: {len(test_samples)}")

#Pipeline for data input
    #prepare image paths
base_image_path = os.path.join(base_path, "words")

def get_image_paths_and_labels(samples):
    paths = []
    corrected_samples = []
    for (i, file_line) in enumerate(samples):
        line_split = file_line.strip()
        line_split = line_split.split(" ")

        # Each line split will have this format for the corresponding image:
        # part1/part1-part2/part1-part2-part3.png
        image_name = line_split[0]
        partI = image_name.split("-")[0]
        partII = image_name.split("-")[1]
        img_path = os.path.join(
            base_image_path, partI, partI + "-" + partII, image_name + ".png"
        )
        if os.path.getsize(img_path):
            paths.append(img_path)
            corrected_samples.append(file_line.split("\n")[0])

    return paths, corrected_samples


train_img_paths, train_labels = get_image_paths_and_labels(train_samples)
validation_img_paths, validation_labels = get_image_paths_and_labels(validation_samples)
test_img_paths, test_labels = get_image_paths_and_labels(test_samples)

    #prepare labels
        # Find maximum length and the size of the vocabulary in the training data.
train_labels_cleaned = []
            #a set includes only distinct features
characters = set()
max_len = 0

for label in train_labels:
    label = label.split(" ")[-1].strip()
    for char in label:
        characters.add(char)

    max_len = max(max_len, len(label))
    train_labels_cleaned.append(label)

characters = sorted(list(characters))

print("Maximum length: ", max_len)
print("Vocab size: ", len(characters))

        #clean validation and test labels
def clean_labels(labels):
    cleaned_labels = []
    for label in labels:
        label = label.split(" ")[-1].strip()
        cleaned_labels.append(label)
    return cleaned_labels

validation_labels_cleaned = clean_labels(validation_labels)
test_labels_cleaned = clean_labels(test_labels)

#Build character Vocabulary
AUTOTUNE = tf.data.AUTOTUNE

    # Mapping characters to integers.
char_to_num = StringLookup(vocabulary=list(characters), mask_token=None)

    # Mapping integers back to original characters.
num_to_char = StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)

#Resize images for batching without distortion
def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    # Check the amount of padding needed to be done.
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    image = tf.pad(image, paddings=[[pad_height, 0],[pad_width, 0],[0, 0],],)

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image

#Applying the defined functions
batch_size = 256
padding_token = 99
image_width = 128
image_height = 32


def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = distortion_free_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image


def vectorize_label(label):
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label


def process_images_labels(image_path, label):
    image = preprocess_image(image_path)
    label = vectorize_label(label)
    return {"image": image, "label": label}


def prepare_dataset(image_paths, labels):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(
        process_images_labels, num_parallel_calls=AUTOTUNE
    )
    return dataset.batch(batch_size).cache().prefetch(AUTOTUNE)

train_ds = prepare_dataset(train_img_paths, train_labels_cleaned)
validation_ds = prepare_dataset(validation_img_paths, validation_labels_cleaned)
test_ds = prepare_dataset(test_img_paths, test_labels_cleaned)

#Implement Layer for CTC Loss
class CTCLayer(keras.layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions.
        return y_pred

class Model(tf.keras.Model):
    def __init__(self, input_shape):
        super().__init__()

        self.loss = CTCLayer()
        self.optimizer = tf.keras.optimizers.Adam()

        #Input layer for predictions later
        #input_img = keras.Input(shape=(128, 32, 1), name="image")
        self.input_layer = tf.keras.layers.InputLayer(input_shape=(128, 32, 1), name="image")

        # input conv1 = 128x32x1
        self.conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer="he_normal", name="Conv1", input_shape=input_shape)
        #ignore this
            #if we use input layer and thus dont specify the input shape here
        #self.conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer="he_normal", name="Conv1")
        
        # output conv1 = 128x32x32
        self.conv2 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer="he_normal", name="Conv2")
        # output conv2 = 128x32x32
        self.pooling1 = MaxPooling2D((2,2), name="pool1")
        # output pooling1 = 64x16x32
        self.conv3 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer="he_normal", name="Conv3")
        # output conv3 = 64x16x64
        self.conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer="he_normal", name="Conv4")
        # output conv3 = 64x16x64
        #self.globalpooling = GlobalAveragePooling2D()
        # output globalpooling = 64
        self.dense1 = Dense(64, activation="relu", name="dense1")
        self.dropout = Dropout(0.1)

        #versteht kein mensch
        self.globalpooling = MaxPooling2D((2,2),name="pool2")
            #output maxpooling: 32x8x64
        new_shape = ((image_width // 4), (image_height // 4) * 64)
        self.reshape = Reshape(target_shape=new_shape, name="reshape")

        #self.lstm1 = keras.layers.LSTM(64, return_sequences=True, dropout=0.15)
        #self.lstm2 = keras.layers.LSTM(64, return_sequences=True, dropout=0.15)

        #what the fuck is this really
        self.lstm1 = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True, dropout=0.1))
        self.lstm2 = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True, dropout=0.1))

        self.output_layer = keras.layers.Dense(len(char_to_num.get_vocabulary()) + 2, activation="softmax", name="dense2")

        self.metrics_list = [tf.keras.metrics.Mean(name="loss")]

    #achtung absatz!!
    @property
    def metrics(self):
        return self.metrics_list

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_state()

    def call(self, x, training = False):
        x = self.input_layer(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pooling1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.globalpooling(x)
        x = self.reshape(x)
        x = self.dense1(x)
        x = self.dropout(x)

        x = self.lstm1(x)
        x = self.lstm2(x)

        output = self.output_layer(x)
        return output

    @tf.function
    def train_step(self, images, label):

        with tf.GradientTape() as tape: 
            prediction = self(images, training = True)
            loss = self.loss(label,prediction)


        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients,self.trainable_variables))
        #self.metrics[0].update_state(label, prediction)
        self.metrics[0].update_state(loss) 

        return {m.name : m.result() for m in self.metrics}  

    @tf.function
    def test_step(self, images, label):

        prediction = self(images, training = False)
        loss = self.loss(label,prediction)
        #self.metrics[0].update_state(label, prediction)
        self.metrics[0].update_state(loss)

        return {m.name : m.result() for m in self.metrics} 

Total training samples: 86810
Total validation samples: 4823
Total test samples: 4823
Maximum length:  21
Vocab size:  78


In [47]:
validation_images = []
validation_labels = []

for batch in validation_ds:
    validation_images.append(batch["image"])
    validation_labels.append(batch["label"])

def calculate_edit_distance(labels, predictions):
    # Get a single batch and convert its labels to sparse tensors.
    saprse_labels = tf.cast(tf.sparse.from_dense(labels), dtype=tf.int64)

    # Make predictions and convert them to sparse tensors.
    input_len = np.ones(predictions.shape[0]) * predictions.shape[1]
    predictions_decoded = keras.backend.ctc_decode(
        predictions, input_length=input_len, greedy=True
    )[0][0][:, :max_len]
    sparse_predictions = tf.cast(
        tf.sparse.from_dense(predictions_decoded), dtype=tf.int64
    )

    # Compute individual edit distances and average them out.
    edit_distances = tf.edit_distance(
        sparse_predictions, saprse_labels, normalize=False
    )
    return tf.reduce_mean(edit_distances)


class EditDistanceCallback(keras.callbacks.Callback):
    def __init__(self, pred_model):
        super().__init__()
        self.prediction_model = pred_model

    def on_epoch_end(self, epoch, logs=None):
        edit_distances = []

        for i in range(len(validation_images)):
            labels = validation_labels[i]
            predictions = self.prediction_model.predict(validation_images[i])
            edit_distances.append(calculate_edit_distance(labels, predictions).numpy())

        tf.summary.scalar('Edit Distance', data=np.mean(edit_distances), step=epoch)


        print(
            f"Mean edit distance for epoch {epoch + 1}: {np.mean(edit_distances):.4f}"
        )



In [48]:
model = Model(input_shape=(128,32,1))
"""
def training_loop(model, train, test, epochs):

    # Save loss and accuracy as dictionaries in a list for visualization
    lists = []

    for n in range(epochs):
        print(f"Epoch {n}:")

        for data in tqdm.tqdm(train, position=0, leave=True):
            images  = data["image"]
            labels  = data["label"]
            metrics = model.train_step(images, labels)

        # Add metrics to list
        lists.append(metrics)
        print([f"{key}: {value.numpy()}" for (key,value) in metrics.items()])
        model.reset_metrics()

        for data in tqdm.tqdm(test, position=0, leave=True):
            images  = data["image"]
            labels  = data["label"]
            metrics = model.test_step(images, labels)

        # Add metrics to list
        lists.append(metrics)
        print([f"{key}: {value.numpy()}" for (key,value) in metrics.items()])
        model.reset_metrics()
        
    return lists

li = training_loop(model, train_ds, test_ds, epochs=15)
"""

'\ndef training_loop(model, train, test, epochs):\n\n    # Save loss and accuracy as dictionaries in a list for visualization\n    lists = []\n\n    for n in range(epochs):\n        print(f"Epoch {n}:")\n\n        for data in tqdm.tqdm(train, position=0, leave=True):\n            images  = data["image"]\n            labels  = data["label"]\n            metrics = model.train_step(images, labels)\n\n        # Add metrics to list\n        lists.append(metrics)\n        print([f"{key}: {value.numpy()}" for (key,value) in metrics.items()])\n        model.reset_metrics()\n\n        for data in tqdm.tqdm(test, position=0, leave=True):\n            images  = data["image"]\n            labels  = data["label"]\n            metrics = model.test_step(images, labels)\n\n        # Add metrics to list\n        lists.append(metrics)\n        print([f"{key}: {value.numpy()}" for (key,value) in metrics.items()])\n        model.reset_metrics()\n        \n    return lists\n\nli = training_loop(model, trai

In [49]:
#prediction_model = model.predict()
prediction_model = keras.models.Model(inputs = model.get_layer(name="image").input, outputs = model.get_layer(name="dense2").output)
edit_distance_callback = EditDistanceCallback(prediction_model)

logdir = "logs/wordRec/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


# Train the model.
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=15,
    callbacks=[tensorboard_callback, edit_distance_callback],
)

AttributeError: ignored